In [15]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import selenium.common.exceptions
from time import sleep
import csv

class mevzuat:
    ELEMENT_LOCATING_TIMEOUT = 60  # in seconds
    
    def __init__(self):
        self.init_driver()
        self.new_file()
        self.bring_legislations()
        while True:
            firstTextElem = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]')))
            firstText = firstTextElem.get_attribute("innerHTML")
                
            i = 1
            while i <= 100:
                self.select_legislation(i)
                self.change_frame()
                self.write_data_to_csv()
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])
                i += 1
            nextPageButton = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#DataTables_Table_0_paginate > ul > li.paginate_button.page-item.active + li > a')))  
            self.safe_click(nextPageButton)
                
            WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until_not(EC.text_to_be_present_in_element_attribute((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]'), "innerHTML", firstText))
                
            self.wait_until_page_fully_loaded()
                
    def init_driver(self):
        # options to open driver
        options = Options()
        options.add_argument('start-maximized')
        options.add_argument('disable-infobars')
        options.add_argument('--disable-extensions')

        # create web driver element
        self.driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Users\Furkan\Desktop\study\nlp\teknofest\notebooks\chromedriver.exe")
        self.URL = 'https://www.mevzuat.gov.tr/#kanunlar'
        self.driver.get(self.URL)
        
    def bring_legislations(self):
        # click on 'ARA' button
        searchElement = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.ID, 'btnSearch')))
        searchElement.click()
        
        self.wait_until_page_fully_loaded()
        
        self.set_listing()
        
    def wait_until_page_fully_loaded(self):
        WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]')))
    
    def safe_click(self, element):
        # safely clicks the given element, meaning loading screen won't get front of the element to be clicked
        try:
            element.click()
        except (selenium.common.exceptions.ElementClickInterceptedException, selenium.common.exceptions.StaleElementReferenceException) as e:
            try:
                sleep(2)
                self.safe_click(element)
            except selenium.common.exceptions.StaleElementReferenceException:
                pass
    
    def set_listing(self):
        # change the listing to 100
        listingSelect = Select(WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '//*[@id="DataTables_Table_0_length"]/label/select'))))
        listingSelect.select_by_value("100")
        
        try:
            WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[100]/td[2]/a/div[1]')))
        except selenium.common.exceptions.TimeoutException:
            pass
        
    def select_legislation(self, i):
        content_xpath = f'//*[@id="DataTables_Table_0"]/tbody/tr[{i}]/td[2]/a'
        link = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        href = link.get_attribute('href')
        self.driver.execute_script("window.open(arguments[0]);", href)
        self.driver.switch_to.window(self.driver.window_handles[1])
        
    def change_frame(self):
        iframe = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))
        self.driver.switch_to.frame(iframe)
        
    def new_file(self):
        newfile = open(r'C:\Users\Furkan\Desktop\study\nlp\teknofest\notebooks\mevzuat.csv', 'w', encoding='utf-8')
        
        header = ["data_text"]
        self.writer = csv.writer(newfile)
        self.writer.writerow(header)
        
    def write_data_to_csv(self):   
        textSegmentElements = self.driver.find_elements_by_xpath('/html/body/div/child::*')
        totalText = ''
        for textSegmentElement in textSegmentElements:
            textSegment = textSegmentElement.text.replace('\n', ' ').strip()
            if len(textSegment) == 0 or textSegment.isdigit():
                continue
            if textSegment[-1] != '.':
                textSegment += '.'
            textSegment += ' '
            totalText += textSegment
        
        self.writer.writerow([totalText])
        
            
mevzuat()

C:\Users\Furkan\AppData\Local\Temp/ipykernel_80384/3771042288.py:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Users\Furkan\Desktop\study\nlp\teknofest\notebooks\chromedriver.exe")
C:\Users\Furkan\AppData\Local\Temp/ipykernel_80384/3771042288.py:48: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Users\Furkan\Desktop\study\nlp\teknofest\notebooks\chromedriver.exe")
C:\Users\Furkan\AppData\Local\Temp/ipykernel_80384/3771042288.py:104: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  textSegmentElements = self.driver.find_elements_by_xpath('/html/body/div/child::*')


TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00A878B3+2193587]
	Ordinal0 [0x00A20681+1771137]
	Ordinal0 [0x009341A8+803240]
	Ordinal0 [0x009624A0+992416]
	Ordinal0 [0x0096273B+993083]
	Ordinal0 [0x0098F7C2+1177538]
	Ordinal0 [0x0097D7F4+1103860]
	Ordinal0 [0x0098DAE2+1170146]
	Ordinal0 [0x0097D5C6+1103302]
	Ordinal0 [0x009577E0+948192]
	Ordinal0 [0x009586E6+952038]
	GetHandleVerifier [0x00D30CB2+2738370]
	GetHandleVerifier [0x00D221B8+2678216]
	GetHandleVerifier [0x00B117AA+512954]
	GetHandleVerifier [0x00B10856+509030]
	Ordinal0 [0x00A2743B+1799227]
	Ordinal0 [0x00A2BB68+1817448]
	Ordinal0 [0x00A2BC55+1817685]
	Ordinal0 [0x00A35230+1856048]
	BaseThreadInitThunk [0x76B1FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76FF7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x76FF7A6E+238]
